# **CIÊNCIA DE DADOS** - DCA3501

UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE, NATAL/RN

DEPARTAMENTO DE ENGENHARIA DE COMPUTAÇÃO E AUTOMAÇÃO

(C) 2025-2026 CARLOS M D VIEGAS

https://github.com/cmdviegas

# VII. Introdução ao **pandas** (parte 2)

Este notebook é a continuação do guia **prático** para estudar a biblioteca **pandas**.

Como vimos anteriormente na parte 1, trabalhamos os **fundamentos do pandas**, aprendendo a:

- Criar e manipular `Series` e `DataFrames`;
- Selecionar e filtrar dados com indexação;
- Tratar valores nulos;
- Converter e ajustar tipos de dados;
- Realizar operações vetorizadas sobre colunas;
- Agrupar e combinar tabelas.

Agora, neste notebook parte 2 sobre o **pandas**, vamos estudar **operações avançadas**, explorando recursos que permitem:

- Aplicar funções personalizadas com `.apply()`;
- Executar transformações em `DataFrames`;
- Trabalhar com hierarquias de índices (*MultiIndex*);
- Introdução a séries temporais;
- Técnicas para otimizar performance e memória.

In [1]:
# Importação das bibliotecas

import pandas as pd
import numpy as np

pd.__version__ # exibe a versão instalada (opcional)

'2.3.2'

## 1. Aplicando funções personalizadas com `.apply()` <a id="sec1"></a>


O método `.apply()` permite aplicar funções às colunas ou linhas de um `DataFrame`.
Isso é útil quando precisamos de maior flexibilidade além das operações vetorizadas.

- `df.apply(func, axis=0)` → aplica a função a cada **coluna**.
- `df.apply(func, axis=1)` → aplica a função a cada **linha**.

**LEMBRETE**: Embora o uso de funções personalizadas com `.apply()` seja útil em situações específicas, é importante lembrar que as operações vetorizadas são, em geral, muito mais rápidas e devem ser a primeira escolha sempre que possível.

In [11]:
# Exemplo de DataFrame
df = pd.DataFrame({
    "A": [10, 20, 30, 40],
    "B": [5, 15, 25, 35],
    "C": [2, 4, 6, 8]
})

df

,A,B,C
0,10,5,2
1,20,15,4
2,30,25,6
3,40,35,8


In [3]:
# Aplicando uma função em cada coluna (axis=0)
df.apply(np.mean)

A    25.0
B    20.0
C     5.0
dtype: float64

In [4]:
# Aplicando função em cada linha (axis=1)
df.apply(lambda row: row["A"] + row["B"] - row["C"], axis=1)

0    13
1    31
2    49
3    67
dtype: int64

In [5]:
# Usando .apply() com funções externas

# Definindo uma função separada
def categorizar_valor(x):
    if x < 15:
        return "baixo"
    elif x < 30:
        return "médio"
    else:
        return "alto"

df["categoria_B"] = df["B"].apply(categorizar_valor)
df


,A,B,C,categoria_B
0,10,5,2,baixo
1,20,15,4,médio
2,30,25,6,médio
3,40,35,8,alto


#### Funções Lambda no Python

As **funções `lambda`** são funções **anônimas** (sem nome) em Python, úteis para criar operações simples e rápidas.
- Não precisam de def nem de nome.
- Normalmente usadas em operações rápidas, quando não vale a pena criar uma função completa.
- Retornam automaticamente o valor da expressão (não é necessário return).

> Sintaxe: 
> ```python
> lambda argumentos: expressão
> ```

In [6]:
# Exemplo de função lambda

# Função tradicional
def quadrado(x):
    return x ** 2

# Equivalente com lambda
quadrado_lambda = lambda x: x ** 2

O **pandas** oferece também métodos semelhantes ao `.apply()` para aplicar funções personalizadas:

- **`.map()`** → usado em **Series** (coluna única). Aplica função a cada elemento.  
- **`.applymap()`** → usado em **DataFrames**. Aplica função a cada elemento da tabela (**em deuso**).  
- **`.apply()`** → já vimos, é o mais flexível (funciona em `Series` e `DataFrames`, por linha ou coluna).

Em resumo:  
- Use `.map()` quando trabalhar com uma única **Series**.  
- Use `.applymap()` quando quiser aplicar função **elemento a elemento** em um **DataFrame**.  
- Use `.apply()` quando precisar de mais flexibilidade (linhas, colunas, funções complexas).


In [8]:
# .map() em Series → atua elemento por elemento em uma única coluna (Series)
# Cada valor é passado para a função e substituído pelo retorno
print(df["A"].map(lambda x: x**2))

# .applymap() em DataFrame → atua elemento por elemento em todo o DataFrame
# Cada célula é passada para a função e substituída pelo retorno
# print(df.applymap(lambda x: f"[{x}]"))

# .apply() em DataFrame → atua em colunas inteiras (padrão, axis=0) ou em linhas inteiras (axis=1)
# print(df.apply(sum, axis=0))

0         7056
1         4489
2           81
3         2601
4         4356
          ... 
999995     529
999996    2209
999997    3969
999998      64
999999    8836
Name: A, Length: 1000000, dtype: int64


In [7]:
# TODO: rever melhor sobre as diferenças de aplicação entre o cálculo vetorizado e o uso de .apply()

# Comparação de desempenho entre operações vetorizadas e apply

# Criando um DataFrame grande
n = 1_000_000 # 1 milhão de linhas
df = pd.DataFrame({"A": np.random.randint(1, 100, size=n)})

# 1. Operação vetorizada
%timeit df["A"] ** 2

# 2. Usando .apply()
%timeit df["A"].apply(lambda x: x ** 2)

1.41 ms ± 64.4 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
247 ms ± 8.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 2. Transformações em DataFrames

Transformações permitem modificar dados **mantendo a estrutura do DataFrame**.

- `.transform(func)` → retorna um objeto do mesmo tamanho do original (mantém o mesmo **número de linhas**)
- Diferença para `.apply()`: `transform` é usado quando queremos retornar valores "transformados" mantendo alinhamento


In [12]:
# Exemplo com transform: padronizando colunas (z-score)
df[["A", "B"]].transform(lambda x: (x - x.mean()) / x.std())

,A,B
0,-1.161895,-1.161895
1,-0.387298,-0.387298
2,0.387298,0.387298
3,1.161895,1.161895


In [13]:
# Usando transform com dicionário
df_scaled = df.transform({"A": lambda x: x * 10, "B": lambda x: x / 5})
df_scaled #.astype("int64")

,A,B
0,100,1.0
1,200,3.0
2,300,5.0
3,400,7.0


In [14]:
df2 = pd.DataFrame({
    "grupo": ["X", "X", "Y", "Y", "Y"],
    "valor": [10, 12, 20, 22, 25]
})

print(df2.groupby("grupo")["valor"].mean())  # média geral

# Subtraindo a média de cada grupo (centralização)
df2["valor_centralizado"] = df2.groupby("grupo")["valor"].transform(lambda x: x - x.mean())
df2


grupo
X    11.000000
Y    22.333333
Name: valor, dtype: float64


,grupo,valor,valor_centralizado
0,X,10,-1.000000
1,X,12,1.000000
2,Y,20,-2.333333
3,Y,22,-0.333333
4,Y,25,2.666667


## 3. Reshape: `pivot`, `melt`, `stack`/`unstack` <a id="12-reshape"></a>

No pandas, **reshape** significa **mudar o formato da tabela** — reorganizando linhas, colunas e índices.

Operações mais comuns:
- `.pivot()` / `.pivot_table()` → transforma dados de formato **longo → largo** (cria colunas novas a partir de valores).
- `.melt()` → transforma dados de formato **largo → longo** (desfaz pivot, "achatando" dados).
- `.stack()` → move colunas para o índice (empilha colunas no índice).
- `.unstack()` → move índices para colunas (faz o oposto, trazendo níveis do índice para colunas).


In [15]:
# Dataframe de referência
df = pd.DataFrame({
    "País": ["Brasil", "Brasil", "Argentina", "Argentina"],
    "Ano": [2020, 2021, 2020, 2021],
    "PIB": [2100, 2200, 450, 470],
    "População": [212, 213, 45, 46]
})
df

,País,Ano,PIB,População
0,Brasil,2020,2100,212
1,Brasil,2021,2200,213
2,Argentina,2020,450,45
3,Argentina,2021,470,46


In [16]:
# Reorganiza os anos em colunas (pivot)
df_pivot = df.pivot(index="País", columns="Ano", values="PIB") # Queremos ver o PIB por País, organizado o ano como colunas
df_pivot

Ano,2020,2021
País,,
Argentina,450,470
Brasil,2100,2200


In [17]:
# Outro exemplo de pivot
df_pivot2 = df.pivot(index="Ano", columns="País", values="População") # Queremos ver a População por Ano, organizando os países como colunas
df_pivot2

País,Argentina,Brasil
Ano,,
2020,45,212
2021,46,213


In [18]:
# Desfaz o pivot: indicadores em uma única coluna (melt)
df_melt = df.melt(id_vars=["País", "Ano"],
                  value_vars=["PIB", "População"],
                  var_name="Indicador",
                  value_name="Valor")
df_melt

,País,Ano,Indicador,Valor
0,Brasil,2020,PIB,2100
1,Brasil,2021,PIB,2200
2,Argentina,2020,PIB,450
3,Argentina,2021,PIB,470
4,Brasil,2020,População,212
5,Brasil,2021,População,213
6,Argentina,2020,População,45
7,Argentina,2021,População,46


In [19]:
# Stack (colunas → índice)
df_stacked = df.set_index(["País", "Ano"]).stack()
df_stacked

País       Ano            
Brasil     2020  PIB          2100
                 População     212
           2021  PIB          2200
                 População     213
Argentina  2020  PIB           450
                 População      45
           2021  PIB           470
                 População      46
dtype: int64

In [20]:
# Unstack (índice → colunas)
df_unstacked = df_stacked.unstack()
df_unstacked

PIB  População
País      Ano                  
Argentina 2020   450         45
          2021   470         46
Brasil    2020  2100        212
          2021  2200        213

## 4. Tabelas dinâmicas (`pivot_table`) <a id="10-pivot"></a>

O `.pivot_table()` funciona de forma semelhante ao `.pivot()`, mas com duas diferenças principais:

- Pode lidar com **valores duplicados**, aplicando uma **função de agregação** (`aggfunc`).
- Por padrão, a função de agregação é a **média** (`mean`).

> Sintaxe básica:
> ```python
> df.pivot_table(index=..., columns=..., values=..., aggfunc=...)
> ```

Em resumo:
- `.pivot()` → reorganiza os dados (não aceita duplicados).
- `.pivot_table()` → além de reorganizar, permite **agregações** sobre dados duplicados.
- `aggfunc` pode ser:
  - Funções simples: `"mean"`, `"sum"`, `"max"`, `"min"`;
  - Uma lista de funções: `["mean", "sum"]`;
  - Ou até uma função customizada.


In [21]:
# Calculando média do PIB (se houver duplicatas)
pivot_pib = df.pivot_table(index="País", columns="Ano", values="PIB", aggfunc="mean")
pivot_pib

Ano,2020,2021
País,,
Argentina,450.0,470.0
Brasil,2100.0,2200.0


In [22]:
# Soma da População por País
pivot_pop = df.pivot_table(index="País", values="População", aggfunc="sum")
pivot_pop

,População
País,
Argentina,91
Brasil,425


In [23]:
# Mais de uma agregação com mais de uma função
pivot_multi = df.pivot_table(index="País", columns="Ano",
                             values="PIB", aggfunc=["mean", "max", "min"])
pivot_multi

mean           max         min      
Ano          2020    2021  2020  2021  2020  2021
País                                             
Argentina   450.0   470.0   450   470   450   470
Brasil     2100.0  2200.0  2100  2200  2100  2200

## 5. Hierarquia de Índices (**MultiIndex**)

Um **MultiIndex** (ou índice hierárquico) permite organizar os dados em **várias dimensões de índice**, não apenas uma.  
Isso é útil quando os dados têm **níveis de hierarquia** — por exemplo, `países → estados → cidades`.

Características principais:
- Um índice pode ter **dois ou mais níveis**.
- Facilita a representação de dados em formato de tabela **multi-dimensional** sem precisar de pivôs complexos.
- Permite operações de seleção e agregação.

In [24]:
# Duas listas de índices
arrays = [
    ["Brasil", "Brasil", "Argentina", "Argentina"],
    ["São Paulo", "Rio de Janeiro", "Buenos Aires", "Córdoba"]
]

index = pd.MultiIndex.from_arrays(arrays, names=("País", "Cidade")) # cria o MultiIndex
print(index)
# Criando DataFrame com MultiIndex
df_multi = pd.DataFrame({
    "População (milhões)": [12.3, 6.7, 15.0, 1.3],
    "PIB (bilhões)": [699, 415, 92, 20]
}, index=index)
# Neste caso, o índice tem dois níveis: País e Cidade

df_multi

MultiIndex([(   'Brasil',      'São Paulo'),
            (   'Brasil', 'Rio de Janeiro'),
            ('Argentina',   'Buenos Aires'),
            ('Argentina',        'Córdoba')],
           names=['País', 'Cidade'])


População (milhões)  PIB (bilhões)
País      Cidade                                            
Brasil    São Paulo                      12.3            699
          Rio de Janeiro                  6.7            415
Argentina Buenos Aires                   15.0             92
          Córdoba                         1.3             20

In [25]:
# Seleção em MultiIndex com .loc()

# Todas as cidades do Brasil
print(df_multi.loc["Brasil"])
print()
# Apenas São Paulo
print(df_multi.loc[("Brasil", "São Paulo")])

                População (milhões)  PIB (bilhões)
Cidade                                            
São Paulo                      12.3            699
Rio de Janeiro                  6.7            415

População (milhões)     12.3
PIB (bilhões)          699.0
Name: (Brasil, São Paulo), dtype: float64


In [26]:
# Seleção por "níveis" do índice com .xs()

# Todas as populações, pegando apenas o nível 'Cidade'
print(df_multi.xs("São Paulo", level="Cidade"))
print()
# Todas as cidades da Argentina, selecionando por nível 'País'
print(df_multi.xs("Argentina", level="País"))

        População (milhões)  PIB (bilhões)
País                                      
Brasil                 12.3            699

              População (milhões)  PIB (bilhões)
Cidade                                          
Buenos Aires                 15.0             92
Córdoba                       1.3             20


In [27]:
# Manipulação de MultiIndex

# Exibindo o DataFrame original
print(df_multi)
print()
# Trocar a ordem dos níveis
df_multi_swapped = df_multi.swaplevel("País", "Cidade")
print(df_multi_swapped)
print()
# Ordenar o índice para facilitar consultas
df_multi_sorted = df_multi.sort_index()
print(df_multi_sorted)


                          População (milhões)  PIB (bilhões)
País      Cidade                                            
Brasil    São Paulo                      12.3            699
          Rio de Janeiro                  6.7            415
Argentina Buenos Aires                   15.0             92
          Córdoba                         1.3             20

                          População (milhões)  PIB (bilhões)
Cidade         País                                         
São Paulo      Brasil                    12.3            699
Rio de Janeiro Brasil                     6.7            415
Buenos Aires   Argentina                 15.0             92
Córdoba        Argentina                  1.3             20

                          População (milhões)  PIB (bilhões)
País      Cidade                                            
Argentina Buenos Aires                   15.0             92
          Córdoba                         1.3             20
Brasil    Rio de Janei

## 6. Séries Temporais

O **pandas** possui suporte nativo para manipulação de **datas e tempos**:

- Criação de faixas de datas com `pd.date_range()`;
- Indexação temporal (ou seja, índices com datas);
- Reamostragem (`resample`) e janelas móveis (`rolling`).


In [28]:
# Criando uma série com datas diárias
datas = pd.date_range("2023-01-01", periods=10, freq="D") # 10 dias a partir de 2023-01-01, freq="D" significa frequência diária
serie = pd.Series(np.random.randint(10, 100, size=10), index=datas) 
serie

2023-01-01    74
2023-01-02    73
2023-01-03    56
2023-01-04    66
2023-01-05    26
2023-01-06    50
2023-01-07    98
2023-01-08    25
2023-01-09    92
2023-01-10    50
Freq: D, dtype: int32

In [29]:
# Selecionar um dia específico
print(serie["2023-01-03"])

# Selecionar um intervalo
print(serie["2023-01-03":"2023-01-06"])

56
2023-01-03    56
2023-01-04    66
2023-01-05    26
2023-01-06    50
Freq: D, dtype: int32


In [30]:
# Reamostragem para frequência semanal (média)
serie.resample("W").mean()

2023-01-01    74.000000
2023-01-08    56.285714
2023-01-15    71.000000
Freq: W-SUN, dtype: float64

In [51]:
# Média móvel de 3 dias
serie.rolling(window=3).mean()


2023-01-01          NaN
2023-01-02          NaN
2023-01-03    42.333333
2023-01-04    48.666667
2023-01-05    39.666667
2023-01-06    47.000000
2023-01-07    52.333333
2023-01-08    68.333333
2023-01-09    85.333333
2023-01-10    92.000000
Freq: D, dtype: float64

In [31]:
# Criando um DataFrame com frequência mensal
datas = pd.date_range("2022-01-01", periods=12, freq="ME")
df_temporal = pd.DataFrame({
    "vendas": np.random.randint(100, 500, size=12),
    "custos": np.random.randint(50, 300, size=12)
}, index=datas)

df_temporal

,vendas,custos
2022-01-31,234,126
2022-02-28,240,292
2022-03-31,481,94
2022-04-30,213,95
2022-05-31,338,149
2022-06-30,248,177
2022-07-31,317,197
2022-08-31,280,55
2022-09-30,454,263
2022-10-31,438,280


#### Operações comuns em Séries Temporais

O pandas oferece métodos poderosos para manipulação e análise de dados temporais.  

- **`resample()`** → muda a granularidade dos dados (ex: diário → semanal, aplicando média, soma etc.).  
- **`diff()`** → calcula a diferença entre períodos consecutivos (ex: crescimento dia a dia).  
- **`shift()`** → desloca valores no tempo (cria defasagens/“lags” ou antecipa valores).  
- **`asfreq()`** → ajusta a frequência da série, inserindo `NaN` ou preenchendo com `ffill` / `bfill`.  


In [32]:
# Operações em séries temporais

# Reamostrando para trimestre (soma)
print(df_temporal.resample("QE").sum())

# Diferença mês a mês
print(df_temporal.diff())

            vendas  custos
2022-03-31     955     512
2022-06-30     799     421
2022-09-30    1051     515
2022-12-31    1177     480
            vendas  custos
2022-01-31     NaN     NaN
2022-02-28     6.0   166.0
2022-03-31   241.0  -198.0
2022-04-30  -268.0     1.0
2022-05-31   125.0    54.0
2022-06-30   -90.0    28.0
2022-07-31    69.0    20.0
2022-08-31   -37.0  -142.0
2022-09-30   174.0   208.0
2022-10-31   -16.0    17.0
2022-11-30  -138.0  -196.0
2022-12-31   139.0    32.0


LEGENDA: Frequências no `resample()`

| Código  | Descrição                          |
|---------|------------------------------------|
| D       | Diário                             |
| B       | Dia útil (business day)            |
| W       | Semanal (final de semana)          |
| ME      | Fim do mês                         |
| MS      | Início do mês                      |
| QE      | Fim do trimestre                   |
| QS      | Início do trimestre                |
| YE      | Fim do ano                         |
| YS      | Início do ano                      |
| H       | Horário (hora em hora)             |
| T, min  | Minuto                             |
| S       | Segundo                            |
| L, ms   | Milissegundo                       |
| U       | Microssegundo                      |
| N       | Nanossegundo                       |


In [33]:
# Deslocamento temporal com .shift()

# Série temporal diária
datas = pd.date_range("2023-01-01", periods=6, freq="D")
serie = pd.Series([10, 20, 15, 25, 30, 40], index=datas)
print("Série original:")
print(serie)

print("\nValor do dia anterior (shift):")
print(serie.shift(1))  # desloca 1 dia para baixo

print("\nDiferença em relação ao dia anterior:")
print(serie - serie.shift(1))

Série original:
2023-01-01    10
2023-01-02    20
2023-01-03    15
2023-01-04    25
2023-01-05    30
2023-01-06    40
Freq: D, dtype: int64

Valor do dia anterior (shift):
2023-01-01     NaN
2023-01-02    10.0
2023-01-03    20.0
2023-01-04    15.0
2023-01-05    25.0
2023-01-06    30.0
Freq: D, dtype: float64

Diferença em relação ao dia anterior:
2023-01-01     NaN
2023-01-02    10.0
2023-01-03    -5.0
2023-01-04    10.0
2023-01-05     5.0
2023-01-06    10.0
Freq: D, dtype: float64


In [34]:
# Frequência com asfreq()

# Criando série mensal
datas_mensais = pd.date_range("2023-01-01", periods=6, freq="ME")
serie_mensal = pd.Series(np.arange(10, 70, 10), index=datas_mensais)
print("Série mensal:")
print(serie_mensal)

# Convertendo para frequência semanal
print("\nReamostrada para frequência semanal, sem preenchimento:")
print(serie_mensal.asfreq("W"))

# Preenchendo valores ausentes para frente (forward fill)
print("\nReamostrada para semanal, preenchendo para frente:")
print(serie_mensal.asfreq("W", method="ffill"))


Série mensal:
2023-01-31    10
2023-02-28    20
2023-03-31    30
2023-04-30    40
2023-05-31    50
2023-06-30    60
Freq: ME, dtype: int64

Reamostrada para frequência semanal, sem preenchimento:
2023-02-05     NaN
2023-02-12     NaN
2023-02-19     NaN
2023-02-26     NaN
2023-03-05     NaN
2023-03-12     NaN
2023-03-19     NaN
2023-03-26     NaN
2023-04-02     NaN
2023-04-09     NaN
2023-04-16     NaN
2023-04-23     NaN
2023-04-30    40.0
2023-05-07     NaN
2023-05-14     NaN
2023-05-21     NaN
2023-05-28     NaN
2023-06-04     NaN
2023-06-11     NaN
2023-06-18     NaN
2023-06-25     NaN
Freq: W-SUN, dtype: float64

Reamostrada para semanal, preenchendo para frente:
2023-02-05    10
2023-02-12    10
2023-02-19    10
2023-02-26    10
2023-03-05    20
2023-03-12    20
2023-03-19    20
2023-03-26    20
2023-04-02    30
2023-04-09    30
2023-04-16    30
2023-04-23    30
2023-04-30    40
2023-05-07    40
2023-05-14    40
2023-05-21    40
2023-05-28    40
2023-06-04    50
2023-06-11    50
20

Resumo:
- `DatetimeIndex` permite usar datas como índice.
- `.resample()` → muda a granularidade temporal (ex: diário → semanal).
- `.rolling()` → janelas móveis (médias móveis, somas acumuladas).
- Métodos como `.shift()`, `.diff()`, `.asfreq()` ampliam a análise temporal.

## 7. Otimizando performance e memória

Algumas dicas importantes para trabalhar com grandes volumes de dados:

- **Tipos de dados mais leves**: converter `int64` → `int32`, `float64` → `float32`.
- **Categorias**: usar `category` para colunas com valores repetidos.
- **Leitura em chunks**: processar arquivos grandes em pedaços.


In [35]:
n = 1000  # número de linhas desejado
df_opt = pd.DataFrame({
    "id": range(1, n + 1),
    "categoria": np.tile(["A", "B", "A", "C", "B"], n // 5)
})
print("Primeiro DF:")
print(df_opt)

print(df_opt.dtypes)
print()

# Criando uma cópia para otimização
df_opt_copia = df_opt.copy()
before = df_opt_copia.memory_usage(deep=True).sum() # Uso de memória antes

# Otimizando
df_opt_copia["id"] = df_opt_copia["id"].astype("int32") # Redução de 64 para 32 bits
df_opt_copia["categoria"] = df_opt_copia["categoria"].astype("category") # Redução com category 
#TODO: procurar pelo tamanho de category

print("Otimizado DF:")
print(df_opt_copia)
print(df_opt_copia.dtypes)
print()
print("Comparação:")
after = df_opt_copia.memory_usage(deep=True).sum() # Uso de memória depois
print(before, after, f"redução={(before-after)/before:.1%}") # Mostra a redução de memória

Primeiro DF:
       id categoria
0       1         A
1       2         B
2       3         A
3       4         C
4       5         B
..    ...       ...
995   996         A
996   997         B
997   998         A
998   999         C
999  1000         B

[1000 rows x 2 columns]
id            int64
categoria    object
dtype: object

Otimizado DF:
       id categoria
0       1         A
1       2         B
2       3         A
3       4         C
4       5         B
..    ...       ...
995   996         A
996   997         B
997   998         A
998   999         C
999  1000         B

[1000 rows x 2 columns]
id              int32
categoria    category
dtype: object

Comparação:
58132 5390 redução=90.7%


In [ ]:
# Leitura em chunks (pedaços)

# Caminho para um arquivo CSV grande
#TODO: criar o arquivo
caminho = "dados_vendas.csv" # precisa criar o arquivo

# Lê em pedaços de 1 mil linhas (por exemplo)
for i, chunk in enumerate(pd.read_csv(caminho, chunksize=1000)):
    print(f"Chunk {i}: {chunk.shape}")
    
    # Exemplo: calcular receita nesse pedaço
    if {"quantidade", "preco_unit"}.issubset(chunk.columns):
        chunk["receita"] = chunk["quantidade"] * chunk["preco_unit"]
        print(chunk[["quantidade", "preco_unit", "receita"]].head())
    
    # Para o exemplo, processa só os 2 primeiros pedaços
    #if i == 1:
    #    break

## 8. `assign` e `pipe`

Até agora aprendemos a aplicar funções customizadas (`apply`, `map`, `transform`), a reorganizar dados (`pivot`, `melt`, `stack`, `unstack`), a trabalhar com **MultiIndex** e com **séries temporais**.  

Agora, vamos ver duas ferramentas que ajudam a **organizar pipelines de transformação** de forma mais legível:

- **`.assign()`** → cria ou modifica colunas no DataFrame.  
- **`.pipe()`** → permite encadear funções externas de forma a estruturar pipelines de transformação.  

In [37]:
# Usando assign para criar colunas derivadas

# Exemplo de DataFrame
df = pd.DataFrame({
    "produto": ["A", "B", "C", "D"],
    "preço": [100, 250, 80, 150],
    "quantidade": [10, 5, 20, 8]
})
print(df)

# Usando assign para criar novas colunas
df2 = (
    df
    .assign(
        receita=lambda x: x["preço"] * x["quantidade"],
        imposto=lambda x: x["preço"] * 0.2,
        receita_liquida=lambda x: x["preço"] * x["quantidade"] - x["preço"] * 0.2
    )
)

print(df2)

  produto  preço  quantidade
0       A    100          10
1       B    250           5
2       C     80          20
3       D    150           8
  produto  preço  quantidade  receita  imposto  receita_liquida
0       A    100          10     1000     20.0            980.0
1       B    250           5     1250     50.0           1200.0
2       C     80          20     1600     16.0           1584.0
3       D    150           8     1200     30.0           1170.0


In [38]:
# Usando pipe para organizar transformações

# Funções externas que transformam o DataFrame
def aplicar_desconto(df, taxa=0.1):
    df = df.copy()
    df["preço_desc"] = df["preço"] * (1 - taxa)
    return df

def filtrar_produtos(df, preco_min=100):
    return df[df["preço"] > preco_min]

# Pipeline com pipe
df_pipe = (
    df
    .pipe(aplicar_desconto, taxa=0.15)
    .pipe(filtrar_produtos, preco_min=90)
)

df_pipe

,produto,preço,quantidade,preço_desc
0,A,100,10,85.0
1,B,250,5,212.5
3,D,150,8,127.5


In [39]:
# Pipeline realista combinando assign + pipe + groupby

# Exemplo de vendas mensais
datas = pd.date_range("2023-01-01", periods=6, freq="ME")
df_vendas = pd.DataFrame({
    "data": datas,
    "produto": ["A", "A", "B", "B", "A", "B"],
    "vendas": [100, 120, 200, 210, 130, 250],
    "preço_unit": [10, 10, 8, 8, 10, 8]
})

# Função externa que agrupa e resume
def resumo_vendas(df):
    return df.groupby("produto").agg(
        receita_total=("receita", "sum"),
        vendas_totais=("vendas", "sum")
    )

# Pipeline completo
df_resultado = (
    df_vendas
    .assign(receita=lambda x: x["vendas"] * x["preço_unit"])
    .pipe(resumo_vendas)
)

df_resultado

,receita_total,vendas_totais
produto,,
A,3500,350
B,5280,660


# Mini-Projeto: Análise de Vendas

Você recebeu um conjunto de dados de vendas de uma rede varejista, contendo:

- **data** → data da venda  
- **loja** → cidade da loja  
- **categoria** → categoria do produto (ex: Eletrônicos, Alimentos, Vestuário)  
- **produto** → nome do produto  
- **quantidade** → quantidade vendida  
- **preço_unit** → preço unitário  

## Objetivo

Responder às seguintes questões usando **pandas** e os conceitos estudados:

1. **Pré-processamento e otimização**  
   - Converter colunas para tipos de dados adequados (`int32`, `category`, `datetime`).  
   - Medir a memória antes e depois (`memory_usage`).  

2. **Criação de colunas derivadas**  
   - Calcular o **faturamento (quantidade × preço_unit)** usando `.assign()`.  
   - Criar uma coluna de **imposto (10% do faturamento)** com `.apply()` ou `.assign()`.  

3. **Transformações e funções customizadas**  
   - Usar `.apply()` para classificar os produtos em "alto ticket" ou "baixo ticket".  
   - Usar `.transform()` para calcular a participação percentual de cada venda dentro da loja.  

4. **Reshape e pivot**  
   - Criar uma tabela **pivot** com o faturamento mensal por categoria.  
   - Usar `.melt()` para voltar ao formato longo.  
   - Explorar `.stack()` e `.unstack()` para reorganizar.  

5. **MultiIndex**  
   - Agrupar os dados por `loja` e `categoria`, calcular o total de faturamento e vendas.  
   - Trabalhar o resultado com índices hierárquicos (selecionar uma loja específica).  

6. **Séries temporais**  
   - Converter `data` para índice (`DatetimeIndex`).  
   - Calcular o faturamento mensal total (`resample("ME").sum()`).  
   - Calcular a variação mês a mês (`diff`) e a média móvel de 3 meses (`rolling`).  
   - Usar `.shift()` para criar uma coluna com o faturamento do mês anterior.  
   - Usar `.asfreq("D")` para converter a série para frequência diária e preencher faltantes (`ffill`).  

7. **Pipeline funcional (`assign` + `pipe`)**  
   - Criar um pipeline que:  
     - Adicione coluna de faturamento,  
     - Filtre apenas a categoria `"Eletrônicos"`,  
     - Resuma os dados por mês,  
     - Retorne um DataFrame final com receita e quantidade mensal. 

>OBSERVAÇÃO: Cada item acima deverá ser respondido em células separadas.  
>OBSERVAÇÃO 2: O código com os dados iniciais é fornecido abaixo.

In [1]:
import pandas as pd
import numpy as np

np.random.seed(42) # para reprodutibilidade (semente fixa de números aleatórios)

datas = pd.date_range("2023-01-01", periods=180, freq="D") # 180 dias
lojas = ["São Paulo", "Rio de Janeiro", "Belo Horizonte"] 
categorias = ["Eletrônicos", "Alimentos", "Vestuário"]
produtos = {
    "Eletrônicos": ["Notebook", "Smartphone", "TV"],
    "Alimentos": ["Arroz", "Feijão", "Macarrão"],
    "Vestuário": ["Camiseta", "Calça", "Tênis"]
}

# Criando DataFrame simulado
df = pd.DataFrame({
    "data": np.random.choice(datas, 1000),
    "loja": np.random.choice(lojas, 1000),
    "categoria": np.random.choice(categorias, 1000),
})

# Preenche produtos de acordo com a categoria
df["produto"] = df["categoria"].map(lambda c: np.random.choice(produtos[c]))
df["quantidade"] = np.random.randint(1, 20, size=1000)
df["preço_unit"] = np.random.randint(20, 2000, size=1000)

# Exibindo as primeiras linhas do DataFrame
df.head()

,data,loja,categoria,produto,quantidade,preço_unit
0,2023-04-13,Belo Horizonte,Vestuário,Calça,8,1594
1,2023-06-29,Belo Horizonte,Alimentos,Feijão,13,1115
2,2023-04-03,São Paulo,Vestuário,Calça,4,1836
3,2023-01-15,Rio de Janeiro,Eletrônicos,Smartphone,8,864
4,2023-04-17,São Paulo,Alimentos,Feijão,3,1715


In [32]:
# Parte 1 - pré-processamento e otimização de memória

# Medição do uso de memória antes da otimização
memoria_antes = df.memory_usage(deep=True).sum()
print(f"Uso de memória antes: {memoria_antes / 1024:.2f} KB")

# Cópia para preservar o original
df_otimizado = df.copy()

# Conversão das colunas para tipos adequados
df_otimizado['data'] = pd.to_datetime(df_otimizado['data'])
df_otimizado['loja'] = df_otimizado['loja'].astype('category')
df_otimizado['categoria'] = df_otimizado['categoria'].astype('category')
df_otimizado['produto'] = df_otimizado['produto'].astype('category')
df_otimizado['quantidade'] = df_otimizado['quantidade'].astype('int32')
df_otimizado['preço_unit'] = df_otimizado['preço_unit'].astype('int32')

# Medição do uso de memória depois da otimização
memoria_depois = df_otimizado.memory_usage(deep=True).sum()
print(f"Uso de memória depois: {memoria_depois / 1024:.2f} KB")

print("\nTipos de dados otimizados:")
df_otimizado.info()

Uso de memória antes: 240.58 KB
Uso de memória depois: 20.35 KB

Tipos de dados otimizados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   data        1000 non-null   datetime64[ns]
 1   loja        1000 non-null   category      
 2   categoria   1000 non-null   category      
 3   produto     1000 non-null   category      
 4   quantidade  1000 non-null   int32         
 5   preço_unit  1000 non-null   int32         
dtypes: category(3), datetime64[ns](1), int32(2)
memory usage: 19.3 KB


In [39]:
# Parte 2 - criando colunas derivadas com assign

# Usando assign() para criar coluna de faturamento e impostoS
df_calculado = df_otimizado.assign(
    faturamento = lambda df_: df_['quantidade'] * df_['preço_unit'],
    imposto = lambda df_: (df_['quantidade'] * df_['preço_unit']) * 0.10
)

print(df_calculado[['data', 'quantidade', 'preço_unit', 'faturamento', 'imposto']].head())

        data  quantidade  preço_unit  faturamento  imposto
0 2023-04-13           8        1594        12752   1275.2
1 2023-06-29          13        1115        14495   1449.5
2 2023-04-03           4        1836         7344    734.4
3 2023-01-15           8         864         6912    691.2
4 2023-04-17           3        1715         5145    514.5


In [40]:
# Parte 3 - Transformações adicionais

# apply() para classificar o ticket
def classificar_ticket(faturamento):
    """Classifica uma venda como 'alto ticket' ou 'baixo ticket'."""
    if faturamento > 10000:
        return "alto ticket"
    return "baixo ticket"

df_transformado = df_calculado.assign(
    tipo_ticket = df_calculado['faturamento'].apply(classificar_ticket)
)

print("Classificação de Ticket:")
print(df_transformado[['faturamento', 'tipo_ticket']].head())

# transform() para calcular a participação percentual
total_faturamento_loja = df_transformado.groupby('loja', observed=True)['faturamento'].transform('sum')
# tive que colocar observed=True para evitar um alerta
df_transformado['participacao_%'] = (df_transformado['faturamento'] / total_faturamento_loja) * 100

print("\nParticipação da venda no total da loja:")
print(df_transformado[['loja', 'faturamento', 'participacao_%']].head())

Classificação de Ticket:
   faturamento   tipo_ticket
0        12752   alto ticket
1        14495   alto ticket
2         7344  baixo ticket
3         6912  baixo ticket
4         5145  baixo ticket

Participação da venda no total da loja:
             loja  faturamento  participacao_%
0  Belo Horizonte        12752        0.343536
1  Belo Horizonte        14495        0.390492
2       São Paulo         7344        0.215617
3  Rio de Janeiro         6912        0.218744
4       São Paulo         5145        0.151055


In [41]:
# Parte 4 - Reshape e pivot

# Criação da tabela pivot
# Extração do mês da data para usar como índice
df_transformado['mes'] = df_transformado['data'].dt.month

tabela_pivot = pd.pivot_table(
    df_transformado,
    values='faturamento',
    index='mes',
    columns='categoria',
    aggfunc='sum',
    fill_value=0, # Preenchimento de nulo pra meses ou categorias sem vendas
    observed=True # Evita o alerta comentado anteriormente
)
print("Faturamento Mensal por Categoria")
print(tabela_pivot)

# melt() para voltar ao formato longo
df_longo = tabela_pivot.reset_index().melt(
    id_vars='mes', 
    var_name='categoria_melt', 
    value_name='faturamento_melt'
)
print("\nDataFrame após melt():")
print(df_longo.head())

# stack() e unstack()
grouped = df_transformado.groupby(['loja', 'categoria'], observed=True)['faturamento'].sum()
print("\nMultiIndex para o unstack:", grouped.head())

unstacked = grouped.unstack(level='categoria')
print("\nDataFrame unstacked:", unstacked)

stacked = unstacked.stack()
print("\nDataFrame stacked:", stacked.head())

Faturamento Mensal por Categoria
categoria  Alimentos  Eletrônicos  Vestuário
mes                                         
1             507138       580740     580260
2             517738       604314     579440
3             459419       568120     514895
4             455515       612061     553814
5             840951       855822     474793
6             516031       501497     555335

DataFrame após melt():
   mes categoria_melt  faturamento_melt
0    1      Alimentos            507138
1    2      Alimentos            517738
2    3      Alimentos            459419
3    4      Alimentos            455515
4    5      Alimentos            840951

MultiIndex para o unstack: loja            categoria  
Belo Horizonte  Alimentos      1137599
                Eletrônicos    1346926
                Vestuário      1227461
Rio de Janeiro  Alimentos       994859
                Eletrônicos    1198803
Name: faturamento, dtype: int32

DataFrame unstacked: categoria       Alimentos  Eletrônicos

In [42]:
# Parte 5 - MultiIndex

# Agrupamento por 'loja' e 'categoria' para criar um MultiIndex
multi_index_df = df_transformado.groupby(['loja', 'categoria'], observed=True).agg(
    faturamento_total=('faturamento', 'sum'),
    vendas_totais=('quantidade', 'sum')
)

print("DataFrame com MultiIndex:")
print(multi_index_df)

# Selecionando dados de uma loja específica usando .loc
print("\nDados de Eletrônicos da loja do Rio de Janeiro:")
print(multi_index_df.loc['Rio de Janeiro', 'Eletrônicos'])

DataFrame com MultiIndex:
                            faturamento_total  vendas_totais
loja           categoria                                    
Belo Horizonte Alimentos              1137599           1207
               Eletrônicos            1346926           1206
               Vestuário              1227461           1179
Rio de Janeiro Alimentos               994859           1128
               Eletrônicos            1198803           1175
               Vestuário               966197           1069
São Paulo      Alimentos              1164334           1134
               Eletrônicos            1176825           1167
               Vestuário              1064879           1004

Dados de Eletrônicos da loja do Rio de Janeiro:
faturamento_total    1198803
vendas_totais           1175
Name: (Rio de Janeiro, Eletrônicos), dtype: int32


In [43]:
# Parte 6 - Séries temporais

# Uso de 'data' para índice
df_ts = df_calculado.set_index('data').sort_index()

# Faturamento mensal total com resample()
faturamento_mensal = df_ts['faturamento'].resample("ME").sum()

# Criação de um DataFrame só pra análise mensal
analise_mensal = pd.DataFrame({'faturamento_mensal': faturamento_mensal})

# Cálculo da variação mês a mês (diff) e a média móvel de 3 meses (rolling)
analise_mensal['variacao'] = analise_mensal['faturamento_mensal'].diff()
analise_mensal['media_movel_3M'] = analise_mensal['faturamento_mensal'].rolling(window=3).mean()

# shift() para ver o faturamento do mês anterior
analise_mensal['mes_anterior'] = analise_mensal['faturamento_mensal'].shift(1)

print("Análise de Faturamento Mensal:")
print(analise_mensal)

# Uso do sfreq("D")
faturamento_diario = df_ts['faturamento'].resample('D').sum()
faturamento_diario_completo = faturamento_diario.asfreq('D').ffill()

print("\nFaturamento diário (com dias faltantes preenchidos):")
print(faturamento_diario_completo.head())

Análise de Faturamento Mensal:
            faturamento_mensal  variacao  media_movel_3M  mes_anterior
data                                                                  
2023-01-31             1668138       NaN             NaN           NaN
2023-02-28             1701492   33354.0             NaN     1668138.0
2023-03-31             1542434 -159058.0    1.637355e+06     1701492.0
2023-04-30             1621390   78956.0    1.621772e+06     1542434.0
2023-05-31             2171566  550176.0    1.778463e+06     1621390.0
2023-06-30             1572863 -598703.0    1.788606e+06     2171566.0

Faturamento diário (com dias faltantes preenchidos):
data
2023-01-01    78079
2023-01-02    46916
2023-01-03    93403
2023-01-04    69532
2023-01-05    41231
Freq: D, Name: faturamento, dtype: int32


In [44]:
# Parte 7 - Pipeline Funcional

# Funções para cada etapa do pipeline
def filtrar_eletronicos(df):
    """Filtra o DataFrame para manter apenas a categoria 'Eletrônicos'."""
    return df[df['categoria'] == 'Eletrônicos']

def resumir_dados_mensais(df):
    """Agrega os dados por mês, calculando receita e quantidade."""
    return (
        df.set_index('data')
        .resample("ME")
        .agg(
            receita_mensal=('faturamento', 'sum'),
            quantidade_mensal=('quantidade', 'sum')
        )
    )

# Criação da pipeline
resultado_pipeline = (
    df_otimizado
    .assign(faturamento=lambda df_: df_['quantidade'] * df_['preço_unit']) # adiciona o faturamento
    .pipe(filtrar_eletronicos)                                             # filtra a categoria
    .pipe(resumir_dados_mensais)                                           # faz o resumo mensal
)

print("Resumo Mensal de Eletrônicos:")
print(resultado_pipeline)

Resumo Mensal de Eletrônicos:
            receita_mensal  quantidade_mensal
data                                         
2023-01-31          580740                576
2023-02-28          604314                520
2023-03-31          568120                516
2023-04-30          612061                681
2023-05-31          855822                754
2023-06-30          501497                501
